# CHAT WITH YOUR DATA - SQL

1. [Google Palm](#Google-Palm)
2. [SQL Database Loader](#SQL-Database-Loader)
3. [Queries](#Queries)
4. [Few Shot Learning](#Few-Shot-Learning)
5. [Vector Stores & Embeddings](#Vector-Stores-&-Embeddings)
6. [Templates](#Templates)
7. [SQL Database Chain Q/A](#SQL-Database-Chain-Q/A)

In [1]:
import os
from pathlib import Path
from dotenv import load_dotenv

env_path = Path('..') / '.env'
_ = load_dotenv(dotenv_path=env_path)

db_credentials = {
    'username': os.getenv('DB_USERNAME'),
    'password': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST'),
}

db_credentials['name'] = 'tshirt_sales'

if not all(db_credentials.values()):
    raise ValueError('One or more environment variables are missing.')

In [2]:
import warnings

warnings.filterwarnings("ignore")

## Google Palm

In [3]:
from langchain.llms import GooglePalm

In [5]:
llm = GooglePalm(google_api_key=os.environ['GOOGLE_API_KEY'], temperature=0)

## SQL Database Loader

In [6]:
from langchain_experimental.sql import SQLDatabaseChain
from langchain_community.utilities.sql_database import SQLDatabase

In [7]:
database_uri = f'mysql+pymysql://{db_credentials["username"]}:{db_credentials["password"]}@{db_credentials["host"]}/{db_credentials["name"]}'

db = SQLDatabase.from_uri(database_uri)

In [8]:
print(f'Dialect: {db.dialect}')
print(f'Table Names: {db.get_usable_table_names()}')

Dialect: mysql
Table Names: ['discounts', 't_shirts']


In [9]:
print(f'Table Info: {db.table_info}')

Table Info: 
CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size

In [10]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

## Queries

In [11]:
# qn1 = db_chain.invoke("How many t-shirts do we have left for nike in extra small size and white color?")

qn1 = db_chain.invoke("SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'")
qn1



> Entering new SQLDatabaseChain chain...
SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'
SQLResult: [(75,)]
Answer:75
> Finished chain.


{'query': "SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
 'result': '75'}

In [12]:
# qn2 = db_chain.invoke("How much is the price of the inventory for all small size t-shirts?")

qn2 = db_chain.invoke("SELECT SUM(PRICE * STOCK_QUANTITY) FROM T_SHIRTS WHERE SIZE = 'S';")
qn2



> Entering new SQLDatabaseChain chain...
SELECT SUM(PRICE * STOCK_QUANTITY) FROM T_SHIRTS WHERE SIZE = 'S';
SQLQuery:SELECT SUM(PRICE * STOCK_QUANTITY) FROM T_SHIRTS WHERE SIZE = 'S'
SQLResult: [(Decimal('17101'),)]
Answer:17101
> Finished chain.


{'query': "SELECT SUM(PRICE * STOCK_QUANTITY) FROM T_SHIRTS WHERE SIZE = 'S';",
 'result': '17101'}

In [13]:
# qn3 = db_chain.invoke("If we have to sell all the Levi’s T-shirts today with discounts applied. \
# How much revenue our store will generate (post discounts)?")

sql_code = """
SELECT SUM(A.TOTAL_AMOUNT * ((100 - COALESCE(DISCOUNTS.PCT_DISCOUNT, 0)) / 100)) AS TOTAL_REVENUE 
FROM
(
    SELECT SUM(PRICE * STOCK_QUANTITY) AS TOTAL_AMOUNT, T_SHIRT_ID 
    FROM T_SHIRTS 
    WHERE BRAND = 'Levi'
    GROUP BY T_SHIRT_ID
) A 
LEFT JOIN DISCOUNTS 
ON A.T_SHIRT_ID = DISCOUNTS.T_SHIRT_ID;
"""

qn3 = db_chain.invoke(sql_code)
qn3



> Entering new SQLDatabaseChain chain...

SELECT SUM(A.TOTAL_AMOUNT * ((100 - COALESCE(DISCOUNTS.PCT_DISCOUNT, 0)) / 100)) AS TOTAL_REVENUE 
FROM
(
    SELECT SUM(PRICE * STOCK_QUANTITY) AS TOTAL_AMOUNT, T_SHIRT_ID 
    FROM T_SHIRTS 
    WHERE BRAND = 'Levi'
    GROUP BY T_SHIRT_ID
) A 
LEFT JOIN DISCOUNTS 
ON A.T_SHIRT_ID = DISCOUNTS.T_SHIRT_ID;

SQLQuery:SELECT SUM(A.TOTAL_AMOUNT * ((100 - COALESCE(DISCOUNTS.PCT_DISCOUNT, 0)) / 100)) AS TOTAL_REVENUE 
FROM
(
    SELECT SUM(PRICE * STOCK_QUANTITY) AS TOTAL_AMOUNT, T_SHIRT_ID 
    FROM T_SHIRTS 
    WHERE BRAND = 'Levi'
    GROUP BY T_SHIRT_ID
) A 
LEFT JOIN DISCOUNTS 
ON A.T_SHIRT_ID = DISCOUNTS.T_SHIRT_ID;
SQLResult: [(Decimal('21453.000000'),)]
Answer:21453.0
> Finished chain.


{'query': "\nSELECT SUM(A.TOTAL_AMOUNT * ((100 - COALESCE(DISCOUNTS.PCT_DISCOUNT, 0)) / 100)) AS TOTAL_REVENUE \nFROM\n(\n    SELECT SUM(PRICE * STOCK_QUANTITY) AS TOTAL_AMOUNT, T_SHIRT_ID \n    FROM T_SHIRTS \n    WHERE BRAND = 'Levi'\n    GROUP BY T_SHIRT_ID\n) A \nLEFT JOIN DISCOUNTS \nON A.T_SHIRT_ID = DISCOUNTS.T_SHIRT_ID;\n",
 'result': '21453.0'}

In [14]:
# qn4 = db_chain.invoke("What is the total value of Levi brand t-shirts in stock?")

qn4 = db_chain.invoke("SELECT SUM(PRICE * STOCK_QUANTITY) FROM T_SHIRTS WHERE BRAND = 'Levi';")
qn4



> Entering new SQLDatabaseChain chain...
SELECT SUM(PRICE * STOCK_QUANTITY) FROM T_SHIRTS WHERE BRAND = 'Levi';
SQLQuery:SELECT SUM(PRICE * STOCK_QUANTITY) FROM T_SHIRTS WHERE BRAND = 'Levi'
SQLResult: [(Decimal('22655'),)]
Answer:22655
> Finished chain.


{'query': "SELECT SUM(PRICE * STOCK_QUANTITY) FROM T_SHIRTS WHERE BRAND = 'Levi';",
 'result': '22655'}

In [15]:
# qn5 = db_chain.invoke("How many white color Levi's t shirts we have available?")

qn5 = db_chain.invoke("SELECT SUM(STOCK_QUANTITY) FROM T_SHIRTS WHERE BRAND = 'Levi' AND COLOR = 'White';")
qn5



> Entering new SQLDatabaseChain chain...
SELECT SUM(STOCK_QUANTITY) FROM T_SHIRTS WHERE BRAND = 'Levi' AND COLOR = 'White';
SQLQuery:SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('204'),)]
Answer:204
> Finished chain.


{'query': "SELECT SUM(STOCK_QUANTITY) FROM T_SHIRTS WHERE BRAND = 'Levi' AND COLOR = 'White';",
 'result': '204'}

## Few Shot Learning

In [16]:
few_shots = [
    {
        'Question': 'How many t-shirts do we have left for Nike in XS size and white color?',
        'SQLQuery': qn1['query'],
        'SQLResult': 'Result of the SQL query',
        'Answer': qn1['result']
    },
    {
        'Question': 'How much is the total price of the inventory for all S-size t-shirts?',
        'SQLQuery': qn2['query'],
        'SQLResult': 'Result of the SQL query',
        'Answer': qn2['result']
    },
    {
        'Question': 'If we have to sell all the Levi’s T-shirts today with discounts applied, how much revenue will our store generate (post discounts)?',
        'SQLQuery': qn3['query'],
        'SQLResult': 'Result of the SQL query',
        'Answer': qn3['result']
    },
    {
        'Question': 'If we have to sell all the Levi’s T-shirts today, how much revenue will our store generate without discount?',
        'SQLQuery': qn4['query'],
        'SQLResult': 'Result of the SQL query',
        'Answer': qn4['result']
    },
    {
        'Question': 'How many white color Levi\'s shirts do I have?',
        'SQLQuery': qn5['query'],
        'SQLResult': 'Result of the SQL query',
        'Answer': qn5['result']
    }
]


## Vector Stores & Embeddings

In [17]:
from langchain.vectorstores import Chroma
from langchain.embeddings import GooglePalmEmbeddings
from langchain.prompts import SemanticSimilarityExampleSelector

In [18]:
to_vectorize = [' '.join(example.values()) for example in few_shots]
to_vectorize[0]

"How many t-shirts do we have left for Nike in XS size and white color? SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS' Result of the SQL query 75"

In [19]:
embeddings = GooglePalmEmbeddings()

vectordb = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots, persist_directory='../data/vector_db/chroma_sql/')

In [20]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectordb,
    k=2,
)

In [21]:
example_selector.select_examples({'Question': 'How many Adidas T shirts I have left in my store?'})

[{'Answer': '75',
  'Question': 'How many t-shirts do we have left for Nike in XS size and white color?',
  'SQLQuery': "SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': '204',
  'Question': "How many white color Levi's shirts do I have?",
  'SQLQuery': "SELECT SUM(STOCK_QUANTITY) FROM T_SHIRTS WHERE BRAND = 'Levi' AND COLOR = 'White';",
  'SQLResult': 'Result of the SQL query'}]

## Templates

In [22]:
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

In [23]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [24]:
print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [25]:
example_prompt = PromptTemplate(
    input_variables=['Question', 'SQLQuery', 'SQLResult','Answer'],
    template='\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}',
)

In [26]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=_mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=['input', 'table_info', 'top_k'],
)

## SQL Database Chain Q/A

In [27]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

In [30]:
new_chain.invoke("How many white color Levi's shirt I have?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's shirt I have?
SQLQuery:SELECT SUM(STOCK_QUANTITY) FROM T_SHIRTS WHERE BRAND = 'Levi' AND COLOR = 'White';
SQLResult: [(Decimal('204'),)]
Answer:204
> Finished chain.


{'query': "How many white color Levi's shirt I have?", 'result': '204'}

In [32]:
new_chain.invoke("How much is the price of all white color levi t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of all white color levi t-shirts?
SQLQuery:SELECT SUM(PRICE * STOCK_QUANTITY) FROM T_SHIRTS WHERE BRAND = 'Levi' AND COLOR = 'White';
SQLResult: [(Decimal('7844'),)]
Answer:7844
> Finished chain.


{'query': 'How much is the price of all white color levi t-shirts?',
 'result': '7844'}

In [33]:
new_chain.invoke("If we have to sell all the Van Heuson T-shirts today with discounts applied. \
How much revenue  our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?
SQLQuery:SELECT SUM(A.TOTAL_AMOUNT * ((100 - COALESCE(DISCOUNTS.PCT_DISCOUNT, 0)) / 100)) AS TOTAL_REVENUE 
FROM
(
    SELECT SUM(PRICE * STOCK_QUANTITY) AS TOTAL_AMOUNT, T_SHIRT_ID 
    FROM T_SHIRTS 
    WHERE BRAND = 'Van Huesen'
    GROUP BY T_SHIRT_ID
) A 
LEFT JOIN DISCOUNTS 
ON A.T_SHIRT_ID = DISCOUNTS.T_SHIRT_ID;
SQLResult: [(Decimal('27755.400000'),)]
Answer:27755.4
> Finished chain.


{'query': 'If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?',
 'result': '27755.4'}

In [34]:
new_chain.invoke("How much revenue  our store will generate by selling all Van Heuson TShirts without discount?")



> Entering new SQLDatabaseChain chain...
How much revenue  our store will generate by selling all Van Heuson TShirts without discount?
SQLQuery:SELECT SUM(PRICE * STOCK_QUANTITY) FROM T_SHIRTS WHERE BRAND = 'Van Huesen';
SQLResult: [(Decimal('28811'),)]
Answer:28811
> Finished chain.


{'query': 'How much revenue  our store will generate by selling all Van Heuson TShirts without discount?',
 'result': '28811'}